In [2]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
#1. Read in Image, Grayscale and Blur
img = cv2.imread('../resources/front3.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
imgBlurred = cv2.GaussianBlur(bfilter, (7, 7), 0)

# sobelX to get the vertical edges
sobelx = cv2.Sobel(imgBlurred, cv2.CV_8U, 1, 0, ksize = 3)
edged = cv2.Canny(sobelx, 30, 200) #Edge detection
# otsu's thresholding
ret2, threshold_img = cv2.threshold(sobelx, 0, 255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
 
element_structure = cv2.getStructuringElement(shape = cv2.MORPH_RECT, ksize=(30, 5))
morph_n_thresholded_img = threshold_img.copy()
cv2.morphologyEx(src = threshold_img,op = cv2.MORPH_CLOSE,kernel = element_structure,dst = morph_n_thresholded_img)

keypoints = cv2.findContours(morph_n_thresholded_img,mode = cv2.RETR_EXTERNAL,method = cv2.CHAIN_APPROX_NONE)
contours = imutils.grab_contours(keypoints)
all_contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
contour_img=img.copy()
cv2.drawContours(contour_img,all_contours,-1,(0,255,0),3)

# find possible plates
possible_plates = []
# minimum area of the plate
min_area = 4500
# maximum area of the plate
max_area = 30000
ratioMin = 2.5
ratioMax = 7
for cnts in all_contours:
    min_rect = cv2.minAreaRect(cnts)
    (x, y), (w, h), rect_angle = min_rect
    if (h == 0 or w == 0):
        continue
    if (w > h):
        angle = -rect_angle
    else:
        angle = 90 + rect_angle
    if angle < 15:
        continue
    
    area = w * h

    ratio = float(w) / float(h)

    if ratio < 1:
        ratio = 1 / ratio
    if (area < min_area or area > max_area) or (ratio < ratioMin or ratio > ratioMax):
        continue
    
    print("x,y,w,h: ",x,y,w,h)
    print("area: ",area)
    print("angle: ",angle)
    print("ratio: ",ratio)
    x, y, w, h = cv2.boundingRect(cnts)
    plate = img[y:y + h, x:x + w]
    cv2.imshow('plate', plate)
    cv2.waitKey()
    cv2.destroyAllWindows()
    possible_plates.append([x,y,w,h])
print(possible_plates)

x,y,w,h:  460.1430969238281 239.0753631591797 49.090965270996094 204.8507843017578
area:  10056.322737893905
angle:  176.8201675415039
ratio:  4.172881571403683
[[358, 209, 205, 61]]
